In [ ]:
!nvidia-smi

Sun Nov  7 20:21:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd /usr/local/cuda/samples/1_Utilities/deviceQuery/
!make
!./deviceQuery

/usr/local/cuda-11.1/samples/1_Utilities/deviceQuery
/usr/local/cuda-11.1/bin/nvcc -ccbin g++ -I../../common/inc  -m64    -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_86,code=compute_86 -o deviceQuery.o -c deviceQuery.cpp
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
/usr/local/cuda-11.1/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=

In [ ]:
%cd /content/
!mkdir workcuda
!cd workcuda/

/content


Crear un fichero y escribir el contenindo

In [ ]:
%%writefile suma1d.cu

#include <stdio.h>
#include <stdlib.h>

__global__ void add(int *a, int *b, int *c) {
*c = *a + *b;
}

int main() {
    
  // host copies of variables a, b & c
  int a, b, c;
  
  // device copies of variables a, b & c  
  int *d_a, *d_b, *d_c;
  
  int size = sizeof(int);

  // Allocate space for device copies of a, b, c
  cudaMalloc((void **)&d_a, size);
  cudaMalloc((void **)&d_b, size);
  cudaMalloc((void **)&d_c, size);

  // Setup input values  
  c = 0;
  a = 3;
  b = 5;

  // Copy inputs to device
  cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);

  // Launch add() kernel on GPU
  add<<<1,1>>>(d_a, d_b, d_c);

  // Copy result back to host
  cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) 
    printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  
  printf("result is %d\n",c);

  // Cleanup
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);

  return 0;
}

Writing suma1d.cu


Compilar

In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true suma1d.cu -o suma1d -lcudadevrt

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [ ]:
!./suma1d

result is 8


Comprobar el perfil de ejecución

In [ ]:
!nvprof ./suma1d

==249== NVPROF is profiling process 249, command: ./suma1d
==249== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
result is 8
==249== Profiling application: ./suma1d
==249== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   37.58%  3.7760us         2  1.8880us  1.5360us  2.2400us  [CUDA memcpy HtoD]
                   36.94%  3.7120us         1  3.7120us  3.7120us  3.7120us  add(int*, int*, int*)
                   25.48%  2.5600us         1  2.5600us  2.5600us  2.5600us  [CUDA memcpy DtoH]
      API calls:   99.59%  248.11ms         3  82.702ms  2.8100us  248.10ms  cudaMalloc
                    0.22%  557.47us         1  557.47us  557.47us  557.47us  cuDeviceTotalMem
                    0.07%  186.28us       101  1.8440us     142ns  75.285us  cuDeviceGetAttribute
                    0.06%  156.75us         3  52.250us  5.4870us  137.77us  cudaFree
                    0.02%  54.267us 

# **PARALELIZAR**

In [ ]:
!rm suma2dvector.cu

rm: cannot remove 'suma2dvector.cu': No such file or directory


# Lanzando N Bloques de 1 Thread

In [ ]:
%%writefile suma2dvector.cu

#include <stdio.h>
#include <stdlib.h>

__global__ void add(int *a, int *b, int *c) {
//*c = *a + *b;
c[blockIdx.x] = a[blockIdx.x] + b[blockIdx.x]; 
}

#define N 2048
#define NUMBLOCKS 65538

int main(void) {
    
  int *a, *b, *c;        // host copies of variables a, b & c
  int *d_a, *d_b, *d_c;  // device copies of variables a, b & c
  int size = N * sizeof(int);

  // Allocate space for host copies of a, b, c   Setup input values  
  a =  (int *) malloc(size); 
  b =  (int *) malloc(size); 
  c =  (int *) malloc(size); 


  //   Setup input values  

  for( int i = 0; i < N; i++ ){
		a[i] = i;
    b[i] = N-i;
		c[i] = 0;
	}


  // Allocate space for device copies of a, b, c
  cudaMalloc((void **)&d_a, size);
  cudaMalloc((void **)&d_b, size);
  cudaMalloc((void **)&d_c, size);


  // Copy inputs to device
  cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

  // Launch add() kernel on GPU  Se lanzan N bloques de 1 Thread.
  add<<<NUMBLOCKS,1>>>(d_a, d_b, d_c);

  // Copy result back to host
  cudaError err = cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
  printf(" valor a[0] es %d\n",a[0]);
  printf(" valor b[0] es %d\n",b[0]);
  printf("resultado c[0] es %d\n",c[0]);
  printf(" valor a[2] es %d\n",a[2]);
  printf(" valor b[2] es %d\n",b[2]);
  printf("resultado c[2] es %d\n",c[2]);

  bool res = 1;
  for(int i  = 0; i < N; i++)
    if(c[i] != N)
      res = 0;

  printf("Todos los valores de c valen lo mismo (1 true, 0 false): %d\n",res);

  // Cleanup

  free(a); free(b);free(c); 

  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);

  return 0;
}

Overwriting suma2dvector.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true suma2dvector.cu -o suma2dvector -lcudadevrt
!./suma2dvector

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
 valor a[0] es 0
 valor b[0] es 2048
resultado c[0] es 2048
 valor a[2] es 2
 valor b[2] es 2046
resultado c[2] es 2048
Todos los valores de c valen lo mismo (1 true, 0 false): 1


Comprobar el perfil de ejecución y compare el resultado con el caso anterior.

In [ ]:
!nvprof ./suma2dvector

==297== NVPROF is profiling process 297, command: ./suma2dvector
==297== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
 valor a[0] es 0
 valor b[0] es 65536
resultado c[0] es 65536
 valor a[2] es 2
 valor b[2] es 65534
resultado c[2] es 65536
Todos los valores de c valen lo mismo (1 true, 0 false): 1
==297== Profiling application: ./suma2dvector
==297== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   70.55%  313.25us         1  313.25us  313.25us  313.25us  add(int*, int*, int*)
                   21.45%  95.232us         2  47.616us  38.432us  56.800us  [CUDA memcpy HtoD]
                    8.01%  35.552us         1  35.552us  35.552us  35.552us  [CUDA memcpy DtoH]
      API calls:   99.24%  195.93ms         3  65.311ms  2.1750us  195.91ms  cudaMalloc
                    0.30%  596.91us         3  198.97us  53.697us  456.90us  cudaMemcpy
                    0.25%  490.02us        

# Lanzando 1 Bloque de N Threads

In [ ]:
%%writefile suma2dvectorNthreads.cu

#include <stdio.h>
#include <stdlib.h>

__global__ void add(int *a, int *b, int *c) {
c[threadIdx.x] = a[threadIdx.x] + b[threadIdx.x]; 
}

#define N 2048

int main(void) {
int *a, *b, *c;        // host copies of variables a, b & c
int *d_a, *d_b, *d_c;  // device copies of variables a, b & c
int size = N * sizeof(int);

// Allocate space for host copies of a, b, c   Setup input values  
a =  (int *) malloc(size); 
b =  (int *) malloc(size); 
c =  (int *) malloc(size); 


// Setup input values  

for( int i = 0; i < N; i++ )
	{
		a[i] = i;
    b[i] = N-i;
		c[i] = 0;
	}


// Allocate space for device copies of a, b, c
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);


// Copy inputs to device
cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

// Launch add() kernel on GPU  Se lanzan 1 bloques de N Threads.
add<<<1,N>>>(d_a, d_b, d_c);

// Copy result back to host
cudaError err = cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
printf(" valor a[10] es %d\n",a[10]);
printf(" valor b[10] es %d\n",b[10]);
printf("resultado c[10] es %d\n",c[10]);
printf(" valor a[0] es %d\n",a[0]);
printf(" valor b[0] es %d\n",b[0]);
printf("resultado c[0] es %d\n",c[0]);


bool res = 1;
int well = 0;
  for(int i  = 0; i < N; i++)
    if(c[i] == N){
      well +=1;
    }

      
res = well == N;

  printf("Todos los valores de c valen lo mismo (1 true, 0 false): %d\n",res);
  printf("Numero de elementos sumados correctamente: %d\n",well);
// Cleanup

free(a); free(b);free(c); 

cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);

return 0;
}

Writing suma2dvectorNthreads.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true suma2dvectorNthreads.cu -o suma2dvectorNthreads -lcudadevrt

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [ ]:
!./suma2dvectorNthreads

 valor a[10] es 10
 valor b[10] es 2038
resultado c[10] es 0
 valor a[0] es 0
 valor b[0] es 2048
resultado c[0] es 0
Todos los valores de c valen lo mismo (1 true, 0 false): 0
Numero de elementos sumados correctamente: 0


Comprobar el perfil de ejecución y compare el resultado con los casos anteriores

In [ ]:
!nvprof ./suma2dvectorNthreads

==344== NVPROF is profiling process 344, command: ./suma2dvectorNthreads
==344== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
 valor a[10] es 10
 valor b[10] es 2038
resultado c[10] es 0
 valor a[0] es 0
 valor b[0] es 2048
resultado c[0] es 0
Todos los valores de c valen lo mismo (1 true, 0 false): 0
Numero de elementos sumados correctamente: 0
==344== Profiling application: ./suma2dvectorNthreads
==344== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   63.69%  6.8480us         2  3.4240us  3.3600us  3.4880us  [CUDA memcpy HtoD]
                   36.31%  3.9040us         1  3.9040us  3.9040us  3.9040us  [CUDA memcpy DtoH]
      API calls:   99.54%  202.26ms         3  67.421ms  2.4890us  202.26ms  cudaMalloc
                    0.25%  498.93us         1  498.93us  498.93us  498.93us  cuDeviceTotalMem
                    0.09%  182.67us       101  1.8080us     143ns  74.404us  cuDe

# Preguntas

* Pruebe a lanzar diferente número de Threads ( con un solo 1 bloque)
¿Cual son los valores máximos y mínimos de número de theads por bloque en esta GPU?

Pruebe a lanzar diferente número de bloques ( con un solo thread) ¿Cual son los valores máximos y mínimos de número de bloques en esta GPU?

#Lanzando NUMBLOQ Bloques de NUMTHREADS Threads

In [ ]:
!rm suma2dvectorNBloqxNthreads.cu

rm: cannot remove 'suma2dvectorNBloqxNthreads.cu': No such file or directory


In [ ]:

%%writefile suma2dvectorNBloqxNthreads.cu

#include <stdio.h>
#include <stdlib.h>

__global__ void add(int *a, int *b, int *c) {
int index = threadIdx.x + blockIdx.x * blockDim.x;
 c[index] = a[index] + b[index]; 
}


#define N (1024*1024 -1)
#define THREADS_PER_BLOCK 512 

int main(void) {
int *a, *b, *c;        // host copies of variables a, b & c
int *d_a, *d_b, *d_c;  // device copies of variables a, b & c
int size = N*sizeof(int);

// Allocate space for host copies of a, b, c   Setup input values  
a =  (int *) malloc(size); 
b =  (int *) malloc(size); 
c =  (int *) malloc(size); 


// Setup input values  

for( int i = 0; i < N; i++ )
	{
		a[i] = i;
    b[i] = N-i;
		c[i] = 0;
	}


// Allocate space for device copies of a, b, c
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);


// Copy inputs to device
cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

// Launch add() kernel on GPU  Se lanzan NuBloq=N/THREADS_PER_BLOCK bloques de 
// THREADS_PER_BLOCK Threads.
add<<<(N + THREADS_PER_BLOCK - 1)/THREADS_PER_BLOCK,THREADS_PER_BLOCK>>>(d_a, d_b, d_c);

// Copy result back to host
cudaError err = cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }

bool res = 1;
int well = 0;
  for(int i  = 0; i < N; i++)
    if(c[i] == N){
      well +=1;
    }

      
res = well == N;

  printf("Todos los valores de c valen lo mismo (1 true, 0 false): %d\n",res);
  printf("Numero de elementos sumados correctamente: %d\n",well);

// Cleanup

free(a); free(b);free(c); 

cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);

return 0;
}

Writing suma2dvectorNBloqxNthreads.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true suma2dvectorNBloqxNthreads.cu -o suma2dvectorNBloqxNthreads -lcudadevrt
!./suma2dvectorNBloqxNthreads

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
Todos los valores de c valen lo mismo (1 true, 0 false): 1
Numero de elementos sumados correctamente: 1048575


In [ ]:
!nvprof ./suma2dvectorNBloqxNthreads

==392== NVPROF is profiling process 392, command: ./suma2dvectorNBloqxNthreads
==392== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
Todos los valores de c valen lo mismo (1 true, 0 false): 1
Numero de elementos sumados correctamente: 1048575
==392== Profiling application: ./suma2dvectorNBloqxNthreads
==392== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   65.81%  1.2620ms         2  631.01us  594.78us  667.23us  [CUDA memcpy HtoD]
                   28.70%  550.40us         1  550.40us  550.40us  550.40us  [CUDA memcpy DtoH]
                    5.49%  105.38us         1  105.38us  105.38us  105.38us  add(int*, int*, int*)
      API calls:   97.79%  201.08ms         3  67.026ms  142.82us  200.74ms  cudaMalloc
                    1.18%  2.4285ms         3  809.49us  749.42us  912.26us  cudaMemcpy
                    0.64%  1.3203ms         3  440.08us  149.00us  594.12us  cudaFree
  

Pregunta ¿Qué sucede si incrementa o disminuye el valor de N?

# SUMA Matrices

In [ ]:

%%writefile sumaMatrices.cu

#include <stdio.h>
#include <stdlib.h>

__global__ void add(int *a, int *b, int *c, int r, int col) {
    int i = (blockIdx.x * blockDim.x) + threadIdx.x;
    int j = (blockIdx.y * blockDim.y) + threadIdx.y;
    int index = col*i + j;

    if (i < r && j < col)
      c[index] = a[index] + b[index]; 
}

#define R 1024
#define C 1014
#define THREADS_PER_BLOCK 8

int main(void) {
int *a, *b, *c;        // host copies of variables a, b & c
int *d_a, *d_b, *d_c;  // device copies of variables a, b & c
int size = R*C*sizeof(int);
// int N = R*C;

// Allocate space for host copies of a, b, c   Setup input values  
a =  (int *) malloc(size); 
b =  (int *) malloc(size); 
c =  (int *) malloc(size); 


// Setup input values  

for( int i = 0; i < R; i++ ){
    for(int j = 0; j < C; j++){
        a[i*C + j] = 2;
        b[i*C + j] = 3;
    }    
}


// Allocate space for device copies of a, b, c
cudaMalloc((void **) &d_a, size);
cudaMalloc((void **) &d_b, size);
cudaMalloc((void **) &d_c, size);


// Copy inputs to device
cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

dim3 dimGrid((R + THREADS_PER_BLOCK - 1)/THREADS_PER_BLOCK, (C + THREADS_PER_BLOCK -1)/THREADS_PER_BLOCK, 1);

dim3 dimBlock(THREADS_PER_BLOCK, THREADS_PER_BLOCK, 1);



add<<<dimGrid,dimBlock>>>(d_a, d_b, d_c,R,C); 

cudaDeviceSynchronize();

// Copy result back to host
cudaError err = cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }


  int errors = 0;
  for(int i = 0; i < R; i++){
      for(int j = 0; j < 1; j++){
          if (c[i*C + j] != 5)
            errors++;
          //printf("%d", d_c[i*C + j]);
      }
  }


  printf("Número de errores: %d\n",errors);


// Cleanup

free(a); free(b);free(c); 

cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);

return 0;
}

Writing sumaMatrices.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true sumaMatrices.cu -o sumaMatrices -lcudadevrt
!./sumaMatrices

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
Número de errores: 0


In [ ]:
!nvprof ./sumaMatrices

==439== NVPROF is profiling process 439, command: ./sumaMatrices
==439== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
Número de errores: 0
==439== Profiling application: ./sumaMatrices
==439== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   53.27%  1.6100ms         1  1.6100ms  1.6100ms  1.6100ms  [CUDA memcpy DtoH]
                   36.96%  1.1170ms         2  558.48us  536.45us  580.51us  [CUDA memcpy HtoD]
                    9.77%  295.20us         1  295.20us  295.20us  295.20us  add(int*, int*, int*, int, int)
      API calls:   96.84%  199.85ms         3  66.618ms  123.89us  199.55ms  cudaMalloc
                    1.96%  4.0386ms         3  1.3462ms  642.32us  2.7361ms  cudaMemcpy
                    0.63%  1.3001ms         3  433.35us  139.38us  584.25us  cudaFree
                    0.23%  478.70us         1  478.70us  478.70us  478.70us  cuDeviceTotalMem
               

In [ ]:
!sed -i '/#define R/c\#define R 512' sumaMatrices.cu
!sed -i '/#define C/c\#define C 1024' sumaMatrices.cu
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true sumaMatrices.cu -o sumaMatrices_changedim -lcudadevrt
!nvprof ./sumaMatrices_changedim

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
==708== NVPROF is profiling process 708, command: ./sumaMatrices_changedim
==708== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
Número de errores: 0
==708== Profiling application: ./sumaMatrices_changedim
==708== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   59.12%  569.54us         2  284.77us  282.66us  286.88us  [CUDA memcpy HtoD]
                   28.03%  270.02us         1  270.02us  270.02us  270.02us  [CUDA memcpy DtoH]
                   12.85%  123.74us         1  123.74us  123.74us  123.74us  add(int*, int*, int*, int, int)
      API calls:   98.16%  203.58ms         3  67.859ms  124.40us  203.27ms  cudaMalloc
                    0.96%  1.9813ms         3  66

In [ ]:
!sed -i '/#define R/c\#define R 111' sumaMatrices.cu
!sed -i '/#define C/c\#define C 333' sumaMatrices.cu
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true sumaMatrices.cu -o sumaMatrices_changedim_2 -lcudadevrt
!nvprof ./sumaMatrices_changedim_2

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
==792== NVPROF is profiling process 792, command: ./sumaMatrices_changedim_2
==792== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
Número de errores: 0
==792== Profiling application: ./sumaMatrices_changedim_2
==792== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   58.69%  46.240us         2  23.120us  22.080us  24.160us  [CUDA memcpy HtoD]
                   26.97%  21.248us         1  21.248us  21.248us  21.248us  [CUDA memcpy DtoH]
                   14.34%  11.296us         1  11.296us  11.296us  11.296us  add(int*, int*, int*, int, int)
      API calls:   99.39%  207.92ms         3  69.306ms  4.0150us  207.91ms  cudaMalloc
                    0.26%  546.53us         1

#EJERCICIO : realice el mismo procedimiento para la ejecución de Stencil1D

Compile el siguiente código, explique su funcionamiento y compruebe si la salida es correcta. En caso de que sea necesario realice las modificaciones oportunas.

In [ ]:
%%writefile stenciltest.cu

#include <stdio.h>

#define RADIUS        3
#define BLOCK_SIZE    256
#define NUM_ELEMENTS  (4096*2)

// CUDA API error checking macro
#define cudaCheck(error) \
  if (error != cudaSuccess) { \
    printf("Fatal error: %s at %s:%d\n", \
      cudaGetErrorString(error), \
      __FILE__, __LINE__); \
    exit(1); \
  }

__global__ void stencil_1d(int *in, int *out) 
{
    //__shared__ int temp[BLOCK_SIZE + 2 * RADIUS];
    int gindex = threadIdx.x + (blockIdx.x * blockDim.x) + RADIUS;
    //int lindex = threadIdx.x + RADIUS;
 

    // Read input elements into shared memory
    //temp[lindex] = in[gindex];
    //if (threadIdx.x < RADIUS) 
    //{
        //temp[lindex - RADIUS] = in[gindex - RADIUS];
        //temp[lindex + BLOCK_SIZE] = in[gindex + BLOCK_SIZE];
    }

    // Make sure all threads get to this point before proceeding!
    //__syncthreads();

    // Apply the stencil
    int result = 0;
    for (int offset = -RADIUS ; offset <= RADIUS ; offset++)
        result += in[gindex + offset];

    // Store the result
    out[gindex-RADIUS] = result;
}

int main()
{
  unsigned int i;
  int h_in[NUM_ELEMENTS + 2 * RADIUS], h_out[NUM_ELEMENTS];
  int *d_in, *d_out;

  // Initialize host data
  for( i = 0; i < (NUM_ELEMENTS + 2*RADIUS); ++i )
    h_in[i] = 1; // With a value of 1 and RADIUS of 3, all output values should be 7

  // Allocate space on the device
  cudaCheck( cudaMalloc( &d_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int)) );
  cudaCheck( cudaMalloc( &d_out, NUM_ELEMENTS * sizeof(int)) );

  // Copy input data to device
  cudaCheck( cudaMemcpy( d_in, h_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int), cudaMemcpyHostToDevice) );

  stencil_1d<<< (NUM_ELEMENTS + BLOCK_SIZE - 1)/BLOCK_SIZE, BLOCK_SIZE >>> (d_in, d_out);

  cudaCheck( cudaMemcpy( h_out, d_out, NUM_ELEMENTS * sizeof(int), cudaMemcpyDeviceToHost) );

  // Verify every out value is 7
  for( i = 0; i < NUM_ELEMENTS; ++i )
    if (h_out[i] != 7)
    {
      printf("Element h_out[%d] == %d != 7\n", i, h_out[i]);
      break;
    }

  if (i == NUM_ELEMENTS)
    printf("SUCCESS!\n");

  // Free out memory
  cudaFree(d_in);
  cudaFree(d_out);

  return 0;
}


Overwriting stenciltest.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true stenciltest.cu -o ./stenciltest -lcudadevrt
!nvprof ./stenciltest

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
stenciltest.cu(20): warning: variable "gindex" was declared but never referenced

stenciltest.cu(37): error: expected a declaration

stenciltest.cu(41): error: identifier "gindex" is undefined

stenciltest.cu(42): error: expected a declaration

stenciltest.cu(74): warning: parsing restarts here after previous syntax error

stenciltest.cu(77): error: this declaration has no storage class or type specifier

stenciltest.cu(77): error: declaration is incompatible with "cudaError_t cudaFree(void *)"
/usr/local/cuda/bin/../targets/x86_64-linux/include/cuda_device_runtime_api.h(145): here

stenciltest.cu(77): error: identifier "d_in" is undefined

stenciltest.cu(78): error: this declaration has no storage class or type specifier

stenciltest.cu(78): error: variable "cudaFree" has al

==169== NVPROF is profiling process 169, command: ./stenciltest
==169== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
==169== Warning: Profiling results might be incorrect with current version of nvcc compiler used to compile cuda app. Compile with nvcc compiler 9.0 or later version to get correct profiling results. Ignore this warning if code is already compiled with the recommended nvcc version 
SUCCESS!
==169== Profiling application: ./stenciltest
==169== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   45.40%  8.9920us         1  8.9920us  8.9920us  8.9920us  [CUDA memcpy HtoD]
                   33.93%  6.7200us         1  6.7200us  6.7200us  6.7200us  [CUDA memcpy DtoH]
                   20.68%  4.0960us         1  4.0960us  4.0960us  4.0960us  stencil_1d(int*, int*)
      API calls:   99.53%  255.49ms         2  127.75ms  5.3530us  255.49ms  cudaMalloc
                    0.1

#EJERCICIO : compile el ejemplo de multiplicación de matrices 

---



In [ ]:
%cd /usr/local/cuda/samples
%ls

In [ ]:
%cd  0_Simple/matrixMul/
%ls


In [ ]:
%pwd



In [ ]:
!cat matrixMul.cu

